In [1]:
import pandas as pd
import os
import glob
import ntpath

In [2]:
workdir = "./2021-08-31/averages/"
dirs = os.listdir(workdir)

In [3]:
mi=pd.MultiIndex.from_tuples

In [4]:
data_dict = {"old": " CogALex 1.0",
            "de_train_new, de_val_new, en_train_new, en_val_new": "CogALex 2.0",
            "de_train_new, de_val_new": " DE 2.0",
            "en_train_new, en_val_new": " EN 2.0"}

baseline = {}

# get baselines per model
for test_avg in glob.glob(workdir + "*/test_avg*.csv"):
    model = ntpath.split(ntpath.split(test_avg)[0])[1].split("_")[0]
    dataset_str = ntpath.split(ntpath.split(test_avg)[0])[1].split("[")[-1].replace("]","").replace("'","")
    if dataset_str not in ["old"]:
        continue
    dataset_list = [data_dict[dataset_str]]
    model_list = [model for i in range(len(dataset_list))]
    tuples= list(zip(*[model_list, dataset_list]))
    df_index = mi(tuples, names=["model","train"])
    results = open(test_avg,"r", encoding="utf-8").readlines()
    temp_dict = {}
    headers = [i.replace("","Language") for i in results[0:-1]]
    for i in results[1:]:
        i = i.split(",")
        temp_dict[i[0]] = [e for e in i[1:-1]]
    baseline[model] = {"DE": float(temp_dict["german"][2]),
                       "EN": float(temp_dict["english"][2]),
                       "ZH": float(temp_dict["chinese"][2]),
                       "IT": float(temp_dict["italian"][2])}

print(baseline)
baseline_list = []
for k,v in baseline.items():
    for i in baseline[k].values():
        baseline_list.append(i)
print(baseline_list)

{'bert-base-multilingual-cased': {'DE': 48.7, 'EN': 57.0, 'ZH': 88.6, 'IT': 48.0}, 'bert-base-multilingual-uncased': {'DE': 53.6, 'EN': 58.5, 'ZH': 88.3, 'IT': 49.0}, 'distilbert-base-multilingual-cased': {'DE': 45.0, 'EN': 52.9, 'ZH': 85.1, 'IT': 40.0}, 'roberta-base': {'DE': 44.8, 'EN': 62.8, 'ZH': 78.5, 'IT': 37.3}, 'xlm-roberta-base': {'DE': 59.8, 'EN': 65.3, 'ZH': 90.6, 'IT': 55.8}}
[48.7, 57.0, 88.6, 48.0, 53.6, 58.5, 88.3, 49.0, 45.0, 52.9, 85.1, 40.0, 44.8, 62.8, 78.5, 37.3, 59.8, 65.3, 90.6, 55.8]


In [5]:
test_df = pd.DataFrame()
for test_avg in glob.glob(workdir + "*/test_avg*.csv"):
    model = ntpath.split(ntpath.split(test_avg)[0])[1].split("_")[0]
    dataset_str = ntpath.split(ntpath.split(test_avg)[0])[1].split("[")[-1].replace("]","").replace("'","")
    if dataset_str in ["de_train_new", "en_train_new", "de_val_new", "en_val_new"]:
        continue
    dataset_list = [data_dict[dataset_str]]
    model_list = [model for i in range(len(dataset_list))]
    tuples= list(zip(*[model_list, dataset_list]))
    df_index = mi(tuples, names=["model","train"])
    results = open(test_avg,"r", encoding="utf-8").readlines()
    temp_dict = {}
    headers = [i.replace("","Language") for i in results[0:-1]]
    for i in results[1:]:
        i = i.split(",")
        temp_dict[i[0]] = [e for e in i[1:-1]]
    if not dataset_str == "old":
        result = {"Test DE": float(temp_dict["german"][2]) - baseline[model]["DE"],
                  "Test EN": float(temp_dict["english"][2]) - baseline[model]["EN"],
                  "Test ZH": float(temp_dict["chinese"][2]) - baseline[model]["ZH"],
                  "Test IT": float(temp_dict["italian"][2]) - baseline[model]["IT"],
                  "Test DE 2.0": float(temp_dict["german NEW"][2]) - baseline[model]["DE"],
                  "Test EN 2.0": float(temp_dict["english NEW"][2]) - baseline[model]["EN"],
                 }
    else:
        result = {"Test DE": float(temp_dict["german"][2]),
                  "Test EN": float(temp_dict["english"][2]),
                  "Test ZH": float(temp_dict["chinese"][2]),
                  "Test IT": float(temp_dict["italian"][2]),
                  "Test DE 2.0": float(temp_dict["german NEW"][2]) - baseline[model]["DE"],
                  "Test EN 2.0": float(temp_dict["english NEW"][2]) - baseline[model]["EN"],
                 }
            
    target_df = pd.DataFrame(result, index=df_index)
    test_df = pd.concat([test_df, target_df])

test_df.sort_index(level=[0,1], ascending=[False,True], inplace=True, sort_remaining=False)

In [6]:
test_df

Test DE  Test EN  Test ZH  \
model                              train                                     
xlm-roberta-base                    CogALex 1.0     59.8     65.3     90.6   
                                    DE 2.0          -0.1     -0.9      0.3   
                                    EN 2.0          -0.6     -0.8      0.8   
                                   CogALex 2.0       0.0     -1.0      1.6   
roberta-base                        CogALex 1.0     44.8     62.8     78.5   
                                   CogALex 2.0       0.8      0.5     -1.7   
distilbert-base-multilingual-cased  CogALex 1.0     45.0     52.9     85.1   
                                   CogALex 2.0       0.3      0.1     -0.4   
bert-base-multilingual-uncased      CogALex 1.0     53.6     58.5     88.3   
                                   CogALex 2.0      -0.8      1.3     -0.3   
bert-base-multilingual-cased        CogALex 1.0     48.7     57.0     88.6   
                                   CogALex 2.0       0.6      1.0     -0.3   

                                                 Test IT  Test DE 2.0  \
model                              train                                
xlm-roberta-base                    CogALex 1.0     55.8          4.8   
                                    DE 2.0           0.0          4.3   
                                    EN 2.0          -0.3          3.2   
                                   CogALex 2.0      -1.5          5.1   
roberta-base                        CogALex 1.0     37.3          0.5   
                                   CogALex 2.0      -3.4          1.5   
distilbert-base-multilingual-cased  CogALex 1.0     40.0          2.2   
                                   CogALex 2.0      -0.8          3.4   
bert-base-multilingual-uncased      CogALex 1.0     49.0          1.3   
                                   CogALex 2.0       0.9          1.8   
bert-base-multilingual-cased        CogALex 1.0     48.0          3.1   
                                   CogALex 2.0      -3.7          4.8   

                                                 Test EN 2.0  
model                              train                      
xlm-roberta-base                    CogALex 1.0          3.8  
                                    DE 2.0               3.8  
                                    EN 2.0               4.6  
                                   CogALex 2.0           4.7  
roberta-base                        CogALex 1.0          2.8  
                                   CogALex 2.0           5.1  
distilbert-base-multilingual-cased  CogALex 1.0          2.9  
                                   CogALex 2.0           2.9  
bert-base-multilingual-uncased      CogALex 1.0          3.2  
                                   CogALex 2.0           5.5  
bert-base-multilingual-cased        CogALex 1.0          2.6  
                                   CogALex 2.0           4.9

In [7]:
def style_negative(v, props=''):
    return props if v < 0 and round(v,1) not in [i for i in baseline_list] else None
def style_positive(v, props=''):
    return props if v > 0 and round(v,1) not in [i for i in baseline_list] else None
def style_baseline(v, props=''):
    return props if v in [i for i in baseline_list] else None

s2 = test_df.style.applymap(style_negative, props='color:red')\
                .applymap(lambda v: 'opacity: 20%;' if (v < 0.3) and (v > -0.3) else None)\
                .applymap(style_baseline, props='font-weight: bold')

s2 = s2.applymap(style_positive, props='color:green;')\
                .applymap(lambda v: 'opacity: 20%;' if (v < 0.3) and (v > -0.3) else None)\
                .applymap(lambda v: 'opacity: 60%;' if (0.3 < v < 2) and (0.3 < v < 2) else None)
print(type(s2))
styled_df = s2.format(precision=1)
styled_df

<class 'pandas.io.formats.style.Styler'>


In [10]:
styled_df.to_latex("./stats/results_on_test.tex", column_format="c", 
                   multirow_align = "c", multicol_align = "c",
                   caption = ("Results","Results on CogALex Test Data"),
                   convert_css = True,
                   label="Evaluation results")